In [4]:
import re
import pickle
import json
import xml.etree.ElementTree as ET
from nltk.corpus import stopwords, wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import os
from nltk.tokenize import word_tokenize, sent_tokenize
import numpy as np
import math
from difflib import SequenceMatcher
import pandas as pd

In [5]:
"""
Section :
Introduction 
Related work
Method
Experiment
Evaluation
Conclusion
"""

'\nSection :\nIntroduction \nRelated work\nMethod\nExperiment\nEvaluation\nConclusion\n'

In [6]:
"""
Existence of any cue word
Number of cue words
subject cue in neighbouring sentences
weight of subject cue 
Location
Popularity
Density
Average Density
Synctatic feature
"""

'\nExistence of any cue word\nNumber of cue words\nsubject cue in neighbouring sentences\nweight of subject cue \nLocation\nPopularity\nDensity\nAverage Density\nSynctatic feature\n'

In [7]:
! pip3 install treetaggerwrapper

In [8]:
citation_sents = pickle.load(open("../pickles/citation_sentences.pkl", "rb"))

In [9]:
cit_strings = pickle.load(open("../pickles/all_cit_strings.pkl", "rb"))

In [10]:
def get_popularity(cit_sents, cit_strings):
    dataset = {}
    for key in cit_sents :
        papers = cit_sents[key]
        pap_list = []
        for pap in papers :
            dict1 = {}
            dict1['paper_name'] = pap['paper_name']
            sents = pap['sents']
            pop = 0
            for sent in sents :
                sent = sent.lower()
                for citstr in cit_strings :
                    if(citstr in sent) :
                        pop+=1
            if(len(sents)>0) :
                pop = pop/len(sents)
            dict1['popularity'] = pop
            pap_list.append(dict1)
        dataset[key] = pap_list
        
    return dataset    

In [13]:
popu = get_popularity(citation_sents, cit_strings)

In [16]:
pickle.dump(popu, open("../pickles/en_popularity.pkl", "wb"))

In [13]:
context = pickle.load(open("../pickles/reduced_context.pkl", "rb"))

In [14]:
def get_density(context, cit_strings):
    dataset = {}
    for key in context :
        papers = context[key]
        pap_list = []
        for pap in papers :
            dict1 = {}
            dict1['paper_name'] = pap['paper_name']
            contexts = pap['context']
            dens = 0
            all_val = []
            for con in contexts :
                con = con.lower()
                for citstr in cit_strings :
                    if(citstr in con) :
                        all_val.append(citstr)
            dens = len(list(set(all_val)))
            if(len(contexts)>0) :
                dens = dens/len(contexts)
            dict1['density'] = dens
            pap_list.append(dict1)
        dataset[key] = pap_list
        
    return dataset    
    
    

In [15]:
dens = get_density(context, cit_strings)
pickle.dump(dens, open("../pickles/en_density.pkl", "wb"))

In [9]:
comp_words = ["compare", "differ", "deviate", "contrast", "exceed", "outperform", "opposed", "consistent with", "significant"]
result_words = ["result", "accuracy", "precision", "performance", "baseline"]
subject_cue = ["we", "our", "us", "table", "figure", "paper", "algorithm", "here"]

In [7]:
stemmer = PorterStemmer()

In [8]:
comp_stem = []
result_stem = []
sub_stem = []
for word in comp_words :
    comp_stem.append(stemmer.stem(word))
    
for word in result_words :
    result_stem.append(stemmer.stem(word))
    
for word in subject_cue :
    sub_stem.append(stemmer.stem(word))

In [14]:

def get_textual_features(context) :
    dataset = {}
    count = 0
    for key in context :
        papers = context[key]
        pap_list = []
        for pap in papers :
            dict1 = {}
            dict1['paper_name'] = pap['paper_name']
            contexts = pap['context']
            dict1['weight_comp'] = 0
            dict1['weight_result'] = 0
            dict1['weight_subject'] = 0
            for con in contexts :
                words = word_tokenize(con)
                for word in words :
                    if(word in comp_words) :
                        dict1['weight_comp']+=1
                    if(word in result_words) :
                        dict1['weight_result']+=1
                    
                    sents = sent_tokenize(con)
                    for sent in sents :
                        if("this_citation" not in sent) :
                            words_sent = word_tokenize(sent)
                            for word in words_sent :
                                if(word in subject_cue) :
                                    dict1['weight_subject']+=1
            
            
            if(len(contexts)>0) :
                dict1['weight_comp'] /= len(context)
                dict1['weight_result'] /= len(context)
                dict1['weight_subject'] /= len(context)
                
            pap_list.append(dict1)
            
        dataset[key] = pap_list
        count+=1
        if(count%100==0) :
            print(count)
        
    return dataset           

In [15]:
text_feat = get_textual_features(context)

100
200
300
400
500
600
700
800
900
1000
1100


In [17]:
pickle.dump(text_feat, open("../pickles/en_textfeat_no_stem.pkl", "wb"))